<span style="font-size:1.1em;">Colab'a Google drive'ı entegre ediyoruz. Kullanılacak olan veriseti Google Drive'da bulunmaktadır</span>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import pandas as pd 

<span style="font-size:1.1em;">Google Drive'ımızın root pathi</span> ```gdrive/My Drive``` <span style="font-size:1.1em;">oluyor. Proje için gerekli verisetini **mbti** adında bir klasör oluşturup içerisine yüklüyoruz. İlgili verisetinin pathi</span> ```gdrive/My Drive/mbti/all_users.csv``` <span style="font-size:1.1em;">oluyor.</span>

In [0]:
df = pd.read_csv("gdrive/My Drive/mbti/eliminated_all_users_v2.csv", sep = ';', header = 0) 

In [6]:
df

,user,entry,type,typeClass,I/E,S/N,T/F,J/P
0,19991991,ekşi itiraf sonunda döndüm dolaştım geldim ken...,ENTJ,analysts,E,N,T,J
1,19991991,selda bağcan bu sesi canlı dinleyebilmek benim...,ENTJ,analysts,E,N,T,J
2,19991991,eski sevgilisinin mutlu olmasını isteyen insan...,ENTJ,analysts,E,N,T,J
3,19991991,veda ederken 2019'a bir not bırakmak farklı ac...,ENTJ,analysts,E,N,T,J
4,19991991,ingiliz aksanı bu aralar özellikle bayıldığım ...,ENTJ,analysts,E,N,T,J
...,...,...,...,...,...,...,...,...
524835,zaimoglu,zlatan ibrahimoviç türkiye de katil olur,ESFJ,sentinels,E,S,F,J
524836,zaimoglu,tarihteki en büyük yalancılar şike operasyonu ...,ESFJ,sentinels,E,S,F,J
524837,zaimoglu,akp-chp koalisyonu hayali koalisyondur,ESFJ,sentinels,E,S,F,J
524838,zaimoglu,trabzonlu insana hamsi diyen fenerbahçe taraft...,ESFJ,sentinels,E,S,F,J


# Pre-Processing

<span style="font-size:1.1em;"> Entrylere ön işleme adımları uygulanır. Bu adımlar:</span>


1. <span style="font-size:1.1em;">Bütün harflerin küçük harf haline getirilmesi</span>


2. <span style="font-size:1.1em;">Entrylerden stop words'ler silinmelidir.</span>


3. <span style="font-size:1.1em;">Web sitelerinin temizlenmesi</span>


4. <span style="font-size:1.1em;">Noktalama işaretlerinin temizlenmesi</span>


5. <span style="font-size:1.1em;">Rakamların temizlenmesi</span>


6. <span style="font-size:1.1em;">Stemming kullanılması</span>


<span style="font-size:1.1em;">    
Temizlenme işlemlerinde empty string yerine space (' ') ile replace edilerek yapılmalıdır. Arından da fazla boşluklar vs trim edilmelidir.

```[+-]?([0-9]*)([.][0-9])?``` regex olarak kullanılacak hem rakamların silinmesinde hem de ondaklı sayıların silinmesinde --> buna gerek kalmadı

</span>


<span style="font-size:1.1em;"> Stop words, drive'da /mbti altındaki stop_words_tr.txt okunarak alınır</span>



In [0]:
file = open("gdrive/My Drive/mbti/stop_words_tr.txt")
stop_word_list = file.read().split('\n')
file.close()

In [8]:
stop_word_list

['a',
 'aa',
 'aaa',
 'aaah',
 'aah',
 'abe',
 'abes',
 'abo',
 'abosa',
 'abov',
 'abow',
 'abu',
 'aç',
 'acaba',
 'acayip',
 'accelerando',
 'acele',
 'aceleten',
 'acep',
 'açık',
 'açıkça',
 'açıkçası',
 'açıktan',
 'acilen',
 'acımasız',
 'acımasızcasına',
 'âciz',
 'âcizane',
 'adamakıllı',
 'adamcasına',
 'adedî',
 'adeta',
 'âdeta',
 'adilane',
 'adına',
 'afedersin',
 'afedersinin',
 'aferin',
 'affettuoso',
 'ağababa',
 'ağabey',
 'ağır',
 'agitato',
 'ağızdan',
 'agucuk',
 'ah',
 'aha',
 'ahacık',
 'ahbap',
 'aheste',
 'ahir',
 'ahiren',
 'ahlaken',
 'ailecek',
 'ait',
 'akabinde',
 'akıbet',
 'akilane',
 'aklen',
 'aklınca',
 'akşamdan',
 'akşamları',
 'akşamleyin',
 'akşamüstü',
 'akşamüzeri',
 'aksine',
 'alabildiğine',
 'alarga',
 'alelacele',
 'alelhesap',
 'alelhusus',
 'alelıtlak',
 'alelumum',
 'alelusul',
 'âlem',
 'alenen',
 'alessabah',
 'aleykümselâm',
 'alicenap',
 'alimallah',
 'âlimane',
 'allahtan',
 'allegretto',
 'allegro',
 'alo',
 'altı',
 'altıncı',
 'a

<span style="font-size:1.1em">Entrylerdeki harfler küçük harf haline getirilir</span>

In [9]:
df['entry'] = df['entry'].str.lower()
df['entry']

0         ekşi itiraf sonunda döndüm dolaştım geldim ken...
1         selda bağcan bu sesi canlı dinleyebilmek benim...
2         eski sevgilisinin mutlu olmasını isteyen insan...
3         veda ederken 2019'a bir not bırakmak farklı ac...
4         ingiliz aksanı bu aralar özellikle bayıldığım ...
                                ...                        
524835            zlatan ibrahimoviç türkiye de katil olur 
524836    tarihteki en büyük yalancılar şike operasyonu ...
524837              akp-chp koalisyonu hayali koalisyondur 
524838    trabzonlu insana hamsi diyen fenerbahçe taraft...
524839    caner taslaman zonguldak taki paneline katılma...
Name: entry, Length: 524840, dtype: object

<span style="font-size:1.1em;">Entrylerden noktalama işaretleri temizlenir.</span>

In [10]:
import string

df['entry'] = df['entry'].apply(lambda x: x.translate(str.maketrans(string.punctuation, ' '*len(string.punctuation))))
df['entry']

0         ekşi itiraf sonunda döndüm dolaştım geldim ken...
1         selda bağcan bu sesi canlı dinleyebilmek benim...
2         eski sevgilisinin mutlu olmasını isteyen insan...
3         veda ederken 2019 a bir not bırakmak farklı ac...
4         ingiliz aksanı bu aralar özellikle bayıldığım ...
                                ...                        
524835            zlatan ibrahimoviç türkiye de katil olur 
524836    tarihteki en büyük yalancılar şike operasyonu ...
524837              akp chp koalisyonu hayali koalisyondur 
524838    trabzonlu insana hamsi diyen fenerbahçe taraft...
524839    caner taslaman zonguldak taki paneline katılma...
Name: entry, Length: 524840, dtype: object

<span style="font-size:1.1em">Entrylerden rakamlar silinir.</span>

In [11]:
df['entry'] = df['entry'].str.replace('\d+', ' ')
df['entry']

0         ekşi itiraf sonunda döndüm dolaştım geldim ken...
1         selda bağcan bu sesi canlı dinleyebilmek benim...
2         eski sevgilisinin mutlu olmasını isteyen insan...
3         veda ederken   a bir not bırakmak farklı acıla...
4         ingiliz aksanı bu aralar özellikle bayıldığım ...
                                ...                        
524835            zlatan ibrahimoviç türkiye de katil olur 
524836    tarihteki en büyük yalancılar şike operasyonu ...
524837              akp chp koalisyonu hayali koalisyondur 
524838    trabzonlu insana hamsi diyen fenerbahçe taraft...
524839    caner taslaman zonguldak taki paneline katılma...
Name: entry, Length: 524840, dtype: object


<span style="font-size:1.1em;">Entrylerden web site linkleri silinir.</span>

In [12]:
df['entry'] = df['entry'].str.replace(r'http\S+', ' ', regex=True)
df['entry'] = df['entry'].str.replace(r'www\S+', '', regex=True)
df['entry']

0         ekşi itiraf sonunda döndüm dolaştım geldim ken...
1         selda bağcan bu sesi canlı dinleyebilmek benim...
2         eski sevgilisinin mutlu olmasını isteyen insan...
3         veda ederken   a bir not bırakmak farklı acıla...
4         ingiliz aksanı bu aralar özellikle bayıldığım ...
                                ...                        
524835            zlatan ibrahimoviç türkiye de katil olur 
524836    tarihteki en büyük yalancılar şike operasyonu ...
524837              akp chp koalisyonu hayali koalisyondur 
524838    trabzonlu insana hamsi diyen fenerbahçe taraft...
524839    caner taslaman zonguldak taki paneline katılma...
Name: entry, Length: 524840, dtype: object

Entrylerden stop words'ler temizlenir.

In [13]:
df['entry'] = df['entry'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_word_list)]))
df['entry']

0         ekşi itiraf döndüm dolaştım geldim kendime çat...
1         selda bağcan sesi dinleyebilmek büyük mutluluk...
2         eski sevgilisinin mutlu olmasını isteyen insan...
3         veda ederken not bırakmak farklı acılar mutlul...
4         ingiliz aksanı aralar bayıldığım konuşan insan...
                                ...                        
524835                     zlatan ibrahimoviç türkiye katil
524836    tarihteki büyük yalancılar şike operasyonu azi...
524837               akp chp koalisyonu hayali koalisyondur
524838    trabzonlu insana hamsi diyen fenerbahçe taraft...
524839    caner taslaman zonguldak paneline katılmaktan ...
Name: entry, Length: 524840, dtype: object

Fazla olan whitespace karakterleri temizlenir.

In [0]:
df['entry'] = df['entry'].replace('\s+', ' ', regex=True)

<span style="font-size:1.1em">Verisetinin stemmera sokmadan önceki ön işlemeden geçirilmiş hali kaydedilir. Buradaki amaç stemmerdan geçirilmiş ve geçirilmemiş halinin başarı açısından karşılaştırılmasının yapılabilmesi.</span>

In [0]:
df.to_csv("gdrive/My Drive/mbti/preprocessed_dataset_no_stemming.csv", sep=';', encoding='utf-8', index=False)

<span style="font-size:1.1em">SnowballStemmer içerisinde Türkçe dili için kullanabileceğimiz TurkishStemmer import edilir. Stemmer ile kelimelerin köklerine ulaşılır. Lemmatization işlemiyle kök bulunduğu zaman elde edilen kökler biçimsel açıdan da doğru olurken (yani mantıksal açıdan da) stemmer ile elde edilen köklerde böyle bir durum söz konusu değildir. Dolayısıyla stemming edilmis hallerine bakıldıgında mantıksal açıdan düzgün sonuclar vermeyebilir.</span>

<span style="font-size:1.1em">Google Colab'a snowballstemmer ile ilgil package'i indirmek için aşağıdaki komutu çalıştırıyoruz.</span>

In [17]:
!pip install snowballstemmer

In [0]:
from snowballstemmer import TurkishStemmer

In [19]:
stemmer = TurkishStemmer()
df['entry']

0         ekşi itiraf döndüm dolaştım geldim kendime çat...
1         selda bağcan sesi dinleyebilmek büyük mutluluk...
2         eski sevgilisinin mutlu olmasını isteyen insan...
3         veda ederken not bırakmak farklı acılar mutlul...
4         ingiliz aksanı aralar bayıldığım konuşan insan...
                                ...                        
524835                     zlatan ibrahimoviç türkiye katil
524836    tarihteki büyük yalancılar şike operasyonu azi...
524837               akp chp koalisyonu hayali koalisyondur
524838    trabzonlu insana hamsi diyen fenerbahçe taraft...
524839    caner taslaman zonguldak paneline katılmaktan ...
Name: entry, Length: 524840, dtype: object

In [0]:
indexes = df.index

for i in range(len(indexes)):
    index = indexes[i]
    entry = df['entry'][index]
    entry_kokler = stemmer.stemWords(entry.split(" "))
    df['entry'][index] = " ".join(entry_kokler)


In [21]:
df['entry']

0         ekşi itiraf dö dolaş gel kendi çat problem ger...
1         selda bağca ses dinleyebilmek büyük mutluluk s...
2         eski sevgili mutlu olma isteye in ayak taş değ...
3         veda eder not bırakmak farklı acı mutluluk yar...
4         ingil aksa ara bayıldık konuşa insa sevimli ge...
                                ...                        
524835                          zla ibrahimoviç türki katil
524836    tarih büyük yalancı şik operasyo aziz yıldır a...
524837                     akp chp koalisyo hayali koalisyo
524838    trabzonlu insa hamsi diye fenerbahçe taraftar ...
524839      caner taslama zonguldak panel katılmak olduk in
Name: entry, Length: 524840, dtype: object

Entrylerden stop words'ler temizlenir. Stemming sonucunda ele edilen bazı cevapler stop words olabilir

In [22]:
df['entry'] = df['entry'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_word_list)]))
df['entry']

0         ekşi itiraf dö dolaş gel çat problem itemiyor ...
1         selda bağca ses dinleyebilmek büyük mutluluk s...
2         eski sevgili mutlu olma isteye in ayak taş değ...
3         veda eder not bırakmak farklı acı mutluluk yar...
4         ingil aksa ara bayıldık konuşa insa sevimli ge...
                                ...                        
524835                          zla ibrahimoviç türki katil
524836    tarih büyük yalancı şik operasyo aziz yıldır a...
524837                     akp chp koalisyo hayali koalisyo
524838    trabzonlu insa hamsi fenerbahçe taraftar mizah...
524839      caner taslama zonguldak panel katılmak olduk in
Name: entry, Length: 524840, dtype: object

Fazla olan whitespace karakterleri temizlenir.

In [0]:
df['entry'] = df['entry'].replace('\s+', ' ', regex=True)

<span style="font-size:1.1em">Stemmera sokulmuş hali kaydedilir.</span>

In [0]:
df.to_csv("gdrive/My Drive/mbti/preprocessed_dataset_with_stemming.csv", sep=';', encoding='utf-8', index=False)